# Copernicus Dataspace

[Documentation](https://documentation.dataspace.copernicus.eu/APIs/OpenSearch.html)


In [1]:
import requests
import geopandas as gpd
import shapely

from utils import get_australia_coords

url = "https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel1/search.json?"

In [27]:
def search(year, next_url=None):
    if next_url is not None:
        response = requests.get(next_url).json()
        if response.get("features") is None:
            # retry...
            response = requests.get(next_url).json()
    else:
        start = f"{year}-01-01T00:00:00Z"
        end = f"{year}-12-31T23:59:59Z"
        coords = get_australia_coords()
        coords_string = ",".join([f"{x[0]} {x[1]}" for x in coords])
        polygon = f"POLYGON(({coords_string}))"

        params = {
            "startDate": start,
            "completionDate": end,
            "maxRecords": 1000,
            "orbitDirection": "DESCENDING",
            "sensorMode": "IW",
            "productType": "IW_GRDH_1S",
            "geometry": polygon,
        }
        response = requests.get(url, params=params).json()
        if response.get("features") is None:
            # retry...
            response = requests.get(next_url).json()

    try:
        for feature in response["features"]:
            yield feature
    except KeyError:
        print(response)
        pass

    try:
        for link in response["properties"]["links"]:
            if link["rel"] == "next":
                yield from search(year, link["href"])
    except KeyError:
        print(response)
        pass

In [28]:
for year in range(2016, 2024):
    response = search(year)
    features = list(response)
    print(year, len(features))

2016 3632
2017 8635
2018 8833
2019 8906
2020 8875
2021 8805
2022 5755
2023 5671


In [6]:
import json

with open("temp.json", "w") as f:
    f.write(json.dumps(features, indent=2))